<a href="https://colab.research.google.com/github/jerekyyro/Faffe/blob/main/FaFFe_koordinaatit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Paikan koordinaattien lisääminen Exceliin paikannimen perusteella

Tätä Google Colab -notebookia voi käyttää paikan koordinaattien tunnistamiseen automaattisesti. Toiminta pohjautuu paikannimet ja niiden koordinaatit sisältävään Exceliin sekä syötteeksi annettaviin havaintoexceleihin. Jos havaintoexcelissä oleva paikannimi löytyy koordinaatit sisältävästä Excelistä, lisätään koordinaatit havaintoexceliin/csv-tiedostoon (kohta 4). Vaihtoehtoisesti, voidaan tuottaa taulukko, jossa havaintojen määrät summataan paikkakohtaisesti (4b).

# Notebookin käyttö:

Notebook sisältää tekstisoluja (kuten tämä osio), suoritettavia Python-koodisoluja (alla oleva harmaana näkyvä solu) sekä koodin tuottamia tulosteita (muodostuvat koodisolujen alle kun niitä suoritetaan).

**Ennen kuin aloitat**, tallenna itsellesi kopio tästä omaan Google Driveen: *File > Save a copy in Drive*.

Tätä notebookia käytetään suorittamalla yksi kerrallaan koodisoluja. Koodi käynnistetään koodisolun vasemmassa laidassa olevasta play-nappulan näkyisestä nappulasta, joka ilmestyy, kun klikkaat koodisolua tai siirrät kursorin hakasulkeissa olevan numeron kohdalle (esim. [1]).

Jos aktivoit tällaisen tekstisolun, pääset pois tuplaklikkaamalla oikealla tai painamalla SHIFT + ENTER. Voit liikkua solujen välillä nuolinäppäimillä.

SHIFT + ENTER:illä voit myös suorittaa aktiivisen *koodisolun*.

Yhden koodisolun suoritus saattaa kestää jopa minuutin, ehkä enemmän.

Jokaisen koodisolun kohdalla kerrotaan, mitä se tekee ja on lisäohjeita.



#1. Koordinaattitiedoston lataus



In [ ]:
from google.colab import files

koordinaatit = files.upload()

koordinaatit = list(koordinaatit.keys())[0]

Saving fi_cities.xlsx to fi_cities.xlsx


# 2. Läpikäytävien tiedostojen lataus

Kun suoritat allaolevan koodin, voit ladata koneeltasi yhden tai useamman xlsx-tiedoston, jotka haluat käydä läpi. Suoritettuasi koodin, paina *Choose files* -nappia.

(Melko hidas!)

In [ ]:
from google.colab import files

uploaded = files.upload()

for file in uploaded:
  print(file)

Saving harmaahaikara_fin.xlsx to harmaahaikara_fin.xlsx
harmaahaikara_fin.xlsx


# 2b. Xlsx-tiedoston aloitusrivi ja muut määritykset.

Alla on oletuksena otsikot sisältäväksi aloitusriviksi asetettu 6, mikä tarkoittaa, että otsikot ovat excel-tiedoston 7. rivillä (rivi 1 -> 0, rivi 2 -> 1, jne.). Näin allaoleva koodi siis ohittaa varsinaista dataa ennen olevat rivit.

Muuta siis allaolevassa koodissa olevaa numeroa tarpeen mukaan.

Huomaa, että otsikkoriviä ennen olevat rivit jäävät pois läpikäydystä excelistä!

Voit myös muuttaa läpikäytävän sarakkeen otsikkoa ja välilehden nimeä.

Voit tulostaa koordinaatit sisältävät taulukot sekä csv- että xlsx-muodoissa (csv-muoto vaaditaan esim. Kepleriin, xlsx:ää taas on helpompi muokata käsin Excelissä.) Tämä määritellään ulos_csv- ja ulos_xlsx -muuttujissa: kirjoita =-merkin perään "True" mikäli haluat että ohjelma tekee taulukon kyseisessä muodossa, tai "False", jos haluat jättää kyseisen muodon tekemättä.  

Muista suorittaa solu ennen kuin menet eteenpäin.

In [ ]:
aloitusrivi = 6

tekstisarake = "Paikkatieto"

valilehti = "Tulokset"


#Tuotettavan taulukon tyypit. Voit laittaa halutessasi molemmat Trueksi:
ulos_csv = True
ulos_xlsx = False


# 4. Tiedostojen läpikäynti.

Mikäli havaintoexcelissä oleva kaupungin nimi löytyy koordinaatti-excelistä, lisätään havaintoexceliin gps-koordinaatit (lat ja lng).


In [ ]:
import pandas as pd

def cleantext(s):
  if type(s) == float:
    return s
  if type(s) == str:
    return s.strip().lower().capitalize()

coords = pd.read_excel(koordinaatit)
coords["city"] = coords["city"].apply(cleantext)


def add_coordinates(df1, df2, df1_place_col, df2_place_col):
    df2[df2_place_col] = df2[df2_place_col].apply(cleantext)
    merged_df = df2.merge(df1[[df1_place_col, 'lat', 'lng']], left_on=df2_place_col, right_on=df1_place_col, how='left')
    merged_df.drop(columns=[df1_place_col], inplace=True)
    return merged_df

def run(filename):
    data = pd.read_excel(filename, sheet_name = valilehti, header = aloitusrivi)
    merged_data = add_coordinates(coords, data, "city", tekstisarake)
    print(merged_data.head())
    filename = "koordinaatit_"+filename

    if ulos_xlsx:
      merged_data.to_excel(filename)
      files.download(filename)

    if ulos_csv:
      csv_file = filename.replace("xlsx", "csv")
      merged_data.to_csv(csv_file, sep=";", encoding="utf-8-sig")
      files.download(csv_file)

#done = []
for file in uploaded:
  run(file)
#  done.append("koordinaatit_"+file)


#for new_file in done:
#  files.download(new_file)

   Paikkatieto Raakapaikkatieto Havainnon PVM         PVM Päänimeke  \
0     Helsinki         Helsinki          1820  1936-07-24   Karjala   
1        Espoo            Espoo          1872  1936-07-24   Karjala   
2     Helsinki         Helsinki  1885 lähtien  1936-07-24   Karjala   
3        Espoo            Espoo  1885 lähtien  1936-07-24   Karjala   
4  Kirkkonummi      Kirkkonummi  1885 lähtien  1936-07-24   Karjala   

  Nimi tekstissä                                         Lisätietoa ISSN  Nro  \
0  harmaahaikara  havainto; ensimm. Kerran Suomessa; pitkä artik...  NaN  NaN   
1  harmaahaikara  havainto; pitkä artikkeli eläintäyttäkjä- ja t...  NaN  NaN   
2  harmaahaikara  nähty 1885 lähtien mm. näillä paikoilla; pitkä...  NaN  NaN   
3  harmaahaikara  nähty 1885 lähtien mm. näillä paikoilla; pitkä...  NaN  NaN   
4  harmaahaikara  nähty 1885 lähtien mm. näillä paikoilla; pitkä...  NaN  NaN   

   Sivu Osumaympäristö                                                URL  \
0   NaN  

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 4 b. Havaintojen summaaminen paikoittain

Alla on toinen vaihtoehto, jossa paikkatiedot summataan paikoittain: yksittäiset paikat tulevat siis riveille, ja niihin liitetään omaan sarakkeeseensa tieto, kuinka monta havaintoa kussakin paikassa on. Täten syntyvää taulukkoa voi hyödyntää, jos haluaa lisätä kartalle pisteitä: havaintomäärä-tietoa voi tällöin hyödyntää pisteen koon tai värin määrittämisessä.

**Huomaa**, että lisätessäsi pisteitä karttaan esim. Keplerissä, kannattaa sellaiset rivit, jotka eivät sisällä paikkatietoja (esim. tyhjä paikannimi tai "X") poistaa tiedostosta, jolloin niiden summat eivät häiritse kartan tekoa.

In [ ]:
import pandas as pd

def cleantext(s):
  if type(s) == float:
    return s
  if type(s) == str:
    return s.strip().lower().capitalize()

coords = pd.read_excel(koordinaatit)
coords["city"] = coords["city"].apply(cleantext)

def aggregate_coordinates(coords_df, obs_df, city, tekstisarake):
    aggregated_df = obs_df[tekstisarake].value_counts().reset_index()
    aggregated_df.columns = [city, 'Havaintojen lkm']

    merged_df = aggregated_df.merge(coords_df, left_on=city, right_on=city, how='left')
    merged_df.drop(columns=['country', 'iso2', 'admin_name', 'capital', 'population', 'population_proper'], inplace=True)

    return merged_df

def run(filename):
    data = pd.read_excel(filename, sheet_name = valilehti, header = aloitusrivi)
    merged_data = aggregate_coordinates(coords, data, "city", tekstisarake)

    print(merged_data.head())
    print(merged_data.tail())
    filename = "koordinaatit_kaupungeittain_"+filename

    if ulos_xlsx:
      merged_data.to_excel(filename)
      files.download(filename)

    if ulos_csv:
      csv_file = filename.replace("xlsx", "csv")
      merged_data.to_csv(csv_file, sep=";", encoding="utf-8-sig")
      files.download(csv_file)

for file in uploaded:
  run(file)



          city  N of Observations      lat      lng
0            X                 44      NaN      NaN
1     Helsinki                 16  60.1708  24.9375
2    Kiuruvesi                  8  63.6500  26.6167
3        Espoo                  7  60.2056  24.6556
4  Kirkkonummi                  6  60.1167  24.4167
           city  N of Observations      lat      lng
56       Pyhtää                  1  60.5000  26.5500
57     Naantali                  1  60.4667  22.0333
58  Kannonkoski                  1      NaN      NaN
59  Kangasniemi                  1  61.9900  26.6417
60      Kustavi                  1      NaN      NaN


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#5. Lopetus

Voit tarkastella Colabiin lataamiasi tiedostoja vasemmalla olevan kansio-kuvakkeen takaa. Kun olet ladatut läpikäydyt tiedostot, kannattaa poistaa tiedostot kaikki xlsx-tiedostot.